In [1]:
import easyocr
import pickle
import cv2
import numpy as np
from PIL import Image, ImageDraw
reader = easyocr.Reader(['vi'])
import matplotlib.pyplot as plt
import time
import imutils
from glob import glob as gl
SHOW_IMAGE= True
font = cv2.FONT_HERSHEY_SIMPLEX

In [2]:
from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg
config = Cfg.load_config_from_name('vgg_transformer')
config['weights'] = 'https://drive.google.com/uc?id=13327Y1tz1ohsm5YZMyXVMPIOjoOA0OaA'
config['cnn']['pretrained']=False
config['device'] = 'cpu'
config['predictor']['beamsearch']=False
detector = Predictor(config)

File exists: C:\Users\tiend\.cache/gdown\https-COLON--SLASH--SLASH-drive.google.com-SLASH-uc-QUESTION-id-EQUAL-13327Y1tz1ohsm5YZMyXVMPIOjoOA0OaA


In [3]:
def auto_canny(image, sigma=0.33):
        # compute the median of the single channel pixel intensities
        v = np.median(image)

        # apply automatic Canny edge detection using the computed median
        lower = int(max(0, (1.0 - sigma) * v))
        upper = int(min(255, (1.0 + sigma) * v))
        edged = cv2.Canny(image, lower, upper)

        # return the edged image
        return edged
    
def get_line_info(line):
    x1,y1,x2,y2 = line
    length = np.sqrt((x1-x2)**2+(y1-y2)**2)
    if x1 == x2:
        return 90, length
    angle = (np.arctan((y2-y1)/(x1-x2))*180/np.pi)
    return angle, length

def document_alignment(img):
    img = cv2.resize(img,  (1500, 1900))
    img = img[:, :, ::-1]
    img_show = img.copy()
    gray = cv2.cvtColor(img_show,cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize = 3)
    kernel = np.ones((5,5),np.uint8)
    edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
    edges = cv2.dilate(edges,kernel,iterations = 1)
    ret2,edges = cv2.threshold(edges,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    edges[:, 0:40] = 0
    edges[:, -40:] = 0
    minLineLength = 50
    maxLineGap = 500
    lines = cv2.HoughLinesP(edges,1, np.pi/360, 200 , minLineLength, maxLineGap)
    h_lines = []
    for line in lines:
        x1,y1,x2,y2 = line[0]
        angle = 90
        angle, length = get_line_info(line[0])
        if abs(angle)>80:
            h_lines.append([angle, length])
            cv2.line(img_show,(x1,y1),(x2,y2),(0,255,0),4)
    
    h_lines = np.array(h_lines)
    arg_max = np.argsort(h_lines[:, 1])
    mean_angle = np.mean(h_lines[arg_max[-3:]][:, 0])
    angle = mean_angle-90
    aligned= imutils.rotate_bound(img, angle)
    
    return aligned, mean_angle, img_show

In [8]:
def show(np_img, name="image", show=SHOW_IMAGE):
    if show:
        Image.fromarray(np_img).show(name)

def draw_image(image, list_box, list_label = False, is_show=False):
    image_c = image.copy()
    stt = 0
    for box in list_box:
        cv2.rectangle(image_c,box[0], box[1], (0,0,255), 2)
        if list_label :
            image_c = cv2.putText(image_c, str(stt), box[0], font,  
                   1, (0, 255, 0), 1, cv2.LINE_AA)
            stt+=1
    if is_show:
        show(image_c)
    return image_c

def convert_to_cv_box(list_box):
    results = []
    for box in list_box:
        box = [(box[0], box[2]), (box[1], box[3])]
        results.append(box)
    return results

def get_hpos(box):
    return (box[1][1]+box[0][1])/2

def get_vpos(box):
    return (box[1][0]+box[0][0])/2

def box_merge(box1, box2):
    p1 = (min(box1[0][0], box2[0][0]), min(box1[0][1], box2[0][1]))
    p2 = (max(box1[1][0], box2[1][0]), max(box1[1][1], box2[1][1]))
    return [p1, p2]

def sort_box_by_hpos(list_box, max_grap=10):
    heights = np.zeros(len(list_box))
    for i in range(len(list_box)):
        heights[i] = get_hpos(list_box[i])
    
    arg_sort = np.argsort(heights)
    list_box_new = []
    for i in range(len(list_box)):
        list_box_new.append(list_box[arg_sort[i]])
    list_box = list_box_new, heights[arg_sort]
    
    return  list_box

def key_matching(list_box, heights):
    results = []
    for i in range(len(list_box)):
        vpos = get_vpos(list_box[i])
        line = []

        if 700< vpos<900:
            values = [list_box[i]]
            hpos = heights[i]

            for j in range(len(list_box)):
                if hpos-16 < heights[j]<hpos+16:
                           line.append(list_box[j])
            for box in line:
                this_vpos = get_vpos(box)
                if this_vpos>vpos+20:
                    values.append(box)
            if len(values) == 3:
                if get_vpos(values[1])>get_vpos(values[2]):
                    values = [values[0], values[2], values[1]]
                results.append(values)
    return  results

def get_area(box):
    return (box[1][0]-box[0][0])*(box[1][1]-box[0][1])

def text_recognition(img):
    img =  Image.fromarray(img)
    s = detector.predict(img)
    return s

def box_to_text(img, box):
    x1, y1= box[0]
    x2, y2 = box[1]
    crop = img[y1:y2, x1:x2]
    return text_recognition(crop)
    
def infor_extract(aligned):
    aligned = cv2.resize(aligned, (1500, 1900))
    horizontal_list, free_list = reader.readtext(aligned, text_threshold = 0.2, link_threshold=0.1, add_margin = 0.2, detect_only=True)
    img_show = aligned.copy()
    horizontal_list1 = convert_to_cv_box(horizontal_list)
    horizontal_list2 = []
    for box in horizontal_list1:
        if get_area(box)>200:
            horizontal_list2.append(box)
    sorted_boxs, heights = sort_box_by_hpos(horizontal_list2, max_grap = 3)
    results = key_matching(sorted_boxs, heights)
    output = {}
    for i in range(len(results)):
        key = box_to_text(aligned, results[i][0])
        val1= box_to_text(aligned, results[i][1])
        val2 = box_to_text(aligned, results[i][2])
        if key.isdigit():
            output[key] = [results[i][0], [results[i][1], val1],[results[i][2], val2]]
            
    keys = output.keys()
    for key in keys:
        key_box = output[key][0]
        cv2.rectangle(aligned, key_box[0], key_box[1], (0,0,255), 2)
        for value in output[key][1:]:
            box = value[0]
            cv2.rectangle(aligned, box[0], box[1], (0,255,0), 2)
    return output, aligned


In [11]:
files = gl("images/*")
img = cv2.imread(files[0], 1)
aligned, mean_angle, img_show = document_alignment(img)
output, aligned = infor_extract(aligned)
show(aligned)
keys = output.keys()
for key in keys:
    print("Mã sô:", key, "Số cuối kỳ:", output[key][1][1], "Số đầu năm:", output[key][2][1])

Mã sô: 100 Số cuối kỳ: 1,231,209,865,239 Số đầu năm: 1,097,379,483,771
Mã sô: 110 Số cuối kỳ: 57,858,877,402 Số đầu năm: 172,247,764,083
Mã sô: 111 Số cuối kỳ: 41,009,938,653 Số đầu năm: 162,958,693,777
Mã sô: 112 Số cuối kỳ: 16,848,938,749 Số đầu năm: 9,289,070,306
Mã sô: 120 Số cuối kỳ: 334,054,946,631 Số đầu năm: 264,299,550,814
Mã sô: 123 Số cuối kỳ: 334,054,946,631 Số đầu năm: 264,299,550,814
Mã sô: 130 Số cuối kỳ: 476,376,268,617 Số đầu năm: 352,653,082,238
Mã sô: 131 Số cuối kỳ: 182,909,298,193 Số đầu năm: 164,300,199,287
Mã sô: 132 Số cuối kỳ: 157,081,844,563 Số đầu năm: 41,957,050,878
Mã sô: 135 Số cuối kỳ: 57,971,293,683 Số đầu năm: 57,973,793,233
Mã sô: 136 Số cuối kỳ: 86,460,716,249 Số đầu năm: 96,468,922,911
Mã sô: 137 Số cuối kỳ: -8,046,884,071 Số đầu năm: -8,046,884,071
Mã sô: 140 Số cuối kỳ: 342,665,675,743 Số đầu năm: 293,907,403,178
Mã sô: 141 Số cuối kỳ: 342,665,675,743 Số đầu năm: 29

In [12]:
print(output)

{'100': [[(740, 316), (790, 348)], [[(923, 313), (1133, 349)], '1,231,209,865,239'], [[(1187, 313), (1393, 349)], '1,097,379,483,771']], '110': [[(740, 352), (790, 384)], [[(955, 349), (1133, 385)], '57,858,877,402'], [[(1207, 349), (1395, 385)], '172,247,764,083']], '111': [[(740, 388), (788, 418)], [[(955, 383), (1133, 419)], '41,009,938,653'], [[(1207, 383), (1395, 421)], '162,958,693,777']], '112': [[(740, 422), (790, 454)], [[(957, 419), (1133, 455)], '16,848,938,749'], [[(1231, 419), (1397, 455)], '9,289,070,306']], '120': [[(740, 458), (790, 490)], [[(941, 455), (1131, 491)], '334,054,946,631'], [[(1205, 455), (1395, 491)], '264,299,550,814']], '123': [[(740, 564), (790, 596)], [[(941, 560), (1129, 596)], '334,054,946,631'], [[(1205, 561), (1395, 597)], '264,299,550,814']], '130': [[(740, 600), (790, 632)], [[(941, 597), (1133, 633)], '476,376,268,617'], [[(1205, 597), (1395, 633)], '352,653,082,238']], '131': [[(739, 633), (789, 669)], [[(943, 631), (1131, 667)], '182,909,298,1